In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import torch
import torchvision.transforms as transforms
import os

image_size = 256

def get_img(path):
    image = Image.open(path)
    totensor = transforms.ToTensor()
    transform=transforms.Compose(
        [
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
        ]
    )
    transformed = transform(image)
    return totensor(transformed)[:3]


def get_imgs(path):
    tensor = []
    imgs = os.listdir(path)
    for img in imgs:
        img_path = os.path.join(path,img)
        tensor.append(get_img(img_path))
    return torch.stack(tensor)

def train_test(tensor,split=0.8):
    idx = int(tensor.shape[0] * split)
    return tensor[:idx],tensor[idx:]

def construct(lsts):
    imgs = torch.concatenate(lsts)
    labels = []
    cnt = 0 
    for val in lsts:
        labels += [cnt for i in range(len(val))]
        cnt += 1
    labels = torch.Tensor(labels)
    return imgs,labels

In [ ]:
import os
from main import segment
from PIL import Image
import imageio
import cv2
from matplotlib import pyplot as plt

BASE = "REDACTED"
imgs = [os.path.join(BASE,img) for img in os.listdir(BASE)]


save = "REDACTED"

cnt = 0
for i in range(len(imgs)):
    image = imageio.imread(os.path.join(BASE,imgs[i]))
    height, width = image.shape[:2]
    w = width//4
    h = height//4
    img = cv2.resize(image, (w,h))
    output,u = segment(img,0.5,300,100)

    vals = {}
    # ! vals[x] = [min_x,max_x,min_y,max_y]

    for y in range(h):
        for x in range(w):
            obj = u.find(y * w + x)
            if obj in vals:
                min_x,max_x,min_y,max_y = vals[obj]
                vals[obj] = [min(x,min_x),max(x,max_x),min(y,min_y),max(y,max_y)]
            else:
                vals[obj] = [x,y,x,y]

    cnt += 1
    file_name = "image_{}.jpg".format(cnt)
    for obj in vals:
        min_x,max_x,min_y,max_y = vals[obj]
        if max_x-min_x > 50 and max_y-min_y > 50:
            obj_img = img[min_y:max_y+1, min_x:max_x+1]
            imageio.imwrite(os.path.join(save,file_name), obj_img)
        else:
            print("no")
            # plt.imshow(obj_img)
            # plt.show()
        

In [ ]:
imageio.imwrite(os.path.join(save,file_name), obj_img)

In [ ]:
obj_img